In [ ]:
import json
import glob
from pathlib import Path, WindowsPath

import pandas as pd
import numpy as np

import re
import pyboat

In [ ]:
# User arguments

pattern = r'mask_tf(\d+)_apoc_cell(\d+)\.json'
port = 5005
input_path = "H:/PROJECTS-03/Pablo/oscillating/ppf005_third_analysis/metadata/raw"
nan_threshold = 0.8
current_socket = 'localhost:8888'

In [ ]:
# Ordering functions

def ordering_function_tf(path):
    """Give the timeframe contained in the name of the json file as an integer.
    Use to sort metadata files"""
    
    pattern = r'mask_tf(\d+)_apoc_cell(\d+)\.json'
    match = re.match(pattern, path.name)
    return int(match.group(1))

def ordering_function_cell(path):
    """Give the cell number contained in the name of the json file as an integer.
    Use to sort metadata files"""

    pattern = r'mask_tf(\d+)_apoc_cell(\d+)\.json'
    match = re.match(pattern, path.name)
    return int(match.group(2))

In [ ]:
def process_subdirectory(subdirectory_path, pattern=pattern, df=df):
    json_files = list(subdirectory_path.glob('mask_tf*_apoc_cell*.json'))
    json_files.sort(key=lambda x: (ordering_function_cell(x), ordering_function_tf(x)))
    cells = {re.match(pattern, file.name).group(2) for file in json_files} # Cell numbers as in the metadata file names

    # Check whether cells is emptyy or metadata has been found, if so populate df with average intenstiy
    if cells:
        for cell in cells:
            print(f'Recording cells {subdirectory.name}+{cell}')
            json_cell = [json_file for json_file in json_files if 'apoc_cell'+cell in json_file.stem]
            tfs = [int((re.match(pattern,json_file.name)).group(1)) for json_file in json_cell]
            tfs.sort()
            for (tf, json_file) in zip(tfs, json_cell):
                with json_file.open('r') as file:
                    data_dict = json.load(file)
                    npixels = data_dict.get('npixels')
                    intensity = data_dict.get('intensity')[1] if 'intensity' in data_dict and len(data_dict['intensity']) > 1 else None
                    column_name = subdirectory_path.name+'_'+cell
                    df.at[tf, column_name] = intensity/npixels
    else:
        print(f'No cell metadata in {subdirectory_path.name}')
    
    return df
    
    # except AttributeError as e:
    #     print(f'No cell metadata in {subdirectory_path.name}: {e}')
    #     pass

In [ ]:
root_folder = Path(input_path)
subdirectories = list(root_folder.glob('*/'))[:-1]

In [ ]:
df = pd.DataFrame()
for subdirectory in subdirectories:
    process_subdirectory(subdirectory, df=df)

In [ ]:
# Filter traces with nan_values defined by nan_threshold

df_cleaned = df.drop(columns=df.columns[df.isna().sum() > nan_threshold*len(df)])

In [ ]:
df_cleaned

In [ ]:
from bokeh.io import output_notebook, show, push_notebook
from bokeh.layouts import column, row
from bokeh.models import Button, ColumnDataSource, CustomJS
from bokeh.plotting import figure, curdoc
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.server.server import Server
from tornado.ioloop import IOLoop
import asyncio


global valid_signals

def modify_doc(doc):
    # Initialize selected_plots as a ColumnDataSource
    selected_plots_source = ColumnDataSource(data=dict(selected_plots=[]))

    # Function to get selected plots in Python
    def get_selected_plots():
        return selected_plots_source.data['selected_plots']

    # Function to create plots and buttons layout
    def create_plots_layout():
        plots = []
        buttons = []

        for col in df_cleaned.columns:
            if col not in get_selected_plots():
                source = ColumnDataSource(data={col: df_cleaned[col], 'x': range(len(df_cleaned))})
                p = figure(width=250, height=250, title=col)
                r = p.line('x', col, source=source, line_width=2, color='navy', alpha=0.8)

                button = Button(label=col, width=60, button_type="success")

                def create_button_callback(plot, column_name, btn):
                    def callback():
                        selected_plots = selected_plots_source.data['selected_plots']
                        if column_name in selected_plots:
                            plot.background_fill_color = 'white'
                            selected_plots.remove(column_name)
                            btn.button_type = 'success'
                        else:
                            plot.background_fill_color = 'rgba(255, 0, 0, 0.1)'
                            selected_plots.append(column_name)
                            btn.button_type = 'danger'
                        selected_plots_source.data = {'selected_plots': selected_plots}  # Update the data source
                        global valid_signals
                        valid_signals = df_cleaned.drop(columns=selected_plots_source.data['selected_plots']).columns
                        # push_notebook()  # Ensure updates are reflected in the notebook
                    return callback

                button.on_click(create_button_callback(p, col, button))

                plots.append(p)
                buttons.append(button)

        # Organize layout
        plot_rows = []
        for i in range(0, len(plots), 5):
            plot_row = plots[i:i+5]
            button_row = buttons[i:i+5]
            plot_rows.append(row(*plot_row, column(*button_row)))

        layout = column(*plot_rows)
        return layout

    # Create the initial layout
    layout = create_plots_layout()

    # Button to print excluded plots
    print_button = Button(label="Print list of excluded plots", width=200, button_type="primary")
    def print_selected_plots():
        print(get_selected_plots())
        # push_notebook()  # Ensure notebook updates
    print_button.on_click(print_selected_plots)

    # Button to rerender without selected plots
    rerender_button = Button(label="Exclude selected plots", width=200, button_type="warning")
    def rerender_plots():
        new_layout = create_plots_layout()
        doc.clear()  # Clear the current document
        doc.add_root(column(new_layout, print_button, rerender_button))
        # push_notebook()
    rerender_button.on_click(rerender_plots)

    # Add the layout and buttons to the current document
    doc.add_root(column(layout, print_button, rerender_button))

# Create the application
app = Application(FunctionHandler(modify_doc))


# Display app
# show(app, notebook_handle=True) - Carefull to push notebook

# Or do it through a server

#  Start the Bokeh server
# server = Server({'/': modify_doc}, port=4996)
#  server.start()

# def show_app():
#     server.io_loop.add_callback(server.show, "/")
#     server.io_loop.start()

# show_app()

#########################
# Integrate with the current Jupyter server -- To be checked
server = Server({'/': modify_doc}, port=port, io_loop=IOLoop.current(), allow_websocket_origin=[current_socket, "localhost:"+str(port)])

async def show_app():
    server.io_loop.add_callback(server.show, "/")
    await server.io_loop.start()

# Integrate with the Jupyter notebook event loop
loop = asyncio.get_event_loop()
if loop.is_running():
    loop.create_task(show_app())
else:
    loop.run_until_complete(show_app())

In [ ]:
# Store valid signals

df_cleaned[valid_signals].to_csv('./valid_signals_ppf005')

In [ ]:
## HERE GOES AN ITERATIVE WAY TO DECIDE WHAT TO DO WITH NAN VALUES -- TO BE DONE ##


In [ ]:
# STOP SERVER AND FREE PORT

server.io_loop.stop()